In [25]:
import pandas as pd
import numpy as np
import iMICE
import MICE

df = pd.read_csv('IMDB-Movie-Data.csv')
df = df.drop(df.columns[[2,3,4,5,6,7,10]],axis = 1)
df = df.dropna(axis=0)

df_1=df.copy()
df_1['MetascoreBinary'] = df['Metascore'].apply(lambda x: 1 if x >= 50 else 0)

X = df_1.iloc[:, [2,3]].values  
y = df_1.iloc[:, 5].values

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

#actual
actual=np.concatenate((y_train,y_test),axis=0)
r1=len(y_train)

#y_test
df_2n = pd.DataFrame({'Metascore':y_test[:]})
df_2n.at[:,'Metascore']=np.nan
y_test_n=df_2n.iloc[:,0].values

y_train_1=np.concatenate((y_train,y_test_n),axis=0)
X_train_1=np.concatenate((X_train,X_test),axis=0)

df_2 = pd.DataFrame({'Metascore':y_train_1[:],'Rating':X_train_1[:,0],'Votes':X_train_1[:,1]})



# KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X_train, y_train)
y_pred = trained_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.6436170212765957
[[26 40]
 [27 95]]


# Random Forest

In [27]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.723404255319149
[[ 33  33]
 [ 19 103]]


# iMICE

In [42]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3i = pd.DataFrame(imice_result, columns = ['Metascore', 'Rating', 'Votes'])
predicti = df_3i['Metascore'].map(lambda x: int(x))

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predicti[r1:]))
print(f1_score(y_test, predicti[r1:]))
print(precision_score(y_test, predicti[r1:]))
print(recall_score(y_test, predicti[r1:]))
print(confusion_matrix(actual[r1:], predicti[r1:]))

0.6648936170212766
0.7789473684210526
0.6809815950920245
0.9098360655737705
[[ 14  52]
 [ 11 111]]


# MICE

In [43]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

print(mice_result.shape)
df_3 = pd.DataFrame(mice_result, columns = ['Age', 'Pclass', 'Fare'])
predict = df_3['Age']

predict = np.array(predict)

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predict[r1:]))
print(f1_score(y_test, predict[r1:]))
print(precision_score(y_test, predict[r1:]))
print(recall_score(y_test, predict[r1:]))

#print(classification_report(y_test, predict[r1:]))
print(confusion_matrix(actual[r1:], predict[r1:])) 


(936, 3)
0.648936170212766
0.7518796992481204
0.6944444444444444
0.819672131147541
[[ 22  44]
 [ 22 100]]
